In [1]:
import csv

import redis

In [ ]:
reference_data_redis_client = redis.Redis(host='10.108.4.36', port=6379, db=0)    # using the read endpoint to not accidentally corrupt the redis by attempting to write to it
trade_history_redis_client = redis.Redis(host='10.75.46.229', port=6379, db=0)    # using the read endpoint to not accidentally corrupt the redis by attempting to write to it

Without multiprocessing. Since the function is so basic, we forgo using multiprocessing since the cost of setting up the multiprocessing may outweigh the potential benefits of using it.

In [3]:
cusips_in_trade_history_redis_but_not_in_reference_data_redis = []
for cusip in trade_history_redis_client.scan_iter():    # scan_iter() is superior to keys() for large numbers of keys because it gives you an iterator you can use rather than trying to load all the keys into memory; https://stackoverflow.com/questions/22255589/get-all-keys-in-redis-database-with-python
    if not reference_data_redis_client.exists(cusip):
        cusips_in_trade_history_redis_but_not_in_reference_data_redis.append(cusip.decode('utf-8'))    # decode with 'utf-8' is necessary since the keys are byte-strings; https://stackoverflow.com/questions/606191/convert-bytes-to-a-string-in-python-3

Write results to CSV.

In [4]:
def write_cusip_list_to_csv(cusip_list, csv_file_name):
    '''Write each CUSIP in `cusip_list` to a new line in the csv at `csv_file_name`.'''
    with open(csv_file_name, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        for cusip in cusip_list:
            csv_writer.writerow([cusip])
    return csv_file_name

In [5]:
write_cusip_list_to_csv(cusips_in_trade_history_redis_but_not_in_reference_data_redis, 'cusips_in_trade_history_redis_but_not_in_reference_data_redis.csv')

'cusips_in_trade_history_redis_but_not_in_reference_data_redis.csv'